In [ ]:
import os
import numpy as np
import pandas as pd
import cv2 as cv2
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from tensorflow.keras.models import Model, load_model
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
pd.set_option('display.max_columns', 80)

In [ ]:
def classify(sdir, csv_path,  model_path, name):
    # read in the csv file
    class_df=pd.read_csv(csv_path)    
    img_height=int(class_df['height'].iloc[0])
    img_width =int(class_df['width'].iloc[0])
    img_size=(img_height, img_width)
    scale=class_df['scale by'].iloc[0]
    scale='img*.00784313-1'
    split=scale.split('-')
    s1=float(split[1])
    s2=float(split[0].split('*')[1])    
    path_list=[]
    paths=os.listdir(sdir)
    for f in paths:
        path_list.append(os.path.join(sdir,f))
    model=load_model(model_path)
    image_count=len(path_list)    
    index_list=[] 
    prob_list=[]
    for i in range (image_count):       
        img=plt.imread(path_list[i]) 
        img=cv2.resize(img, img_size) 
        img=img*s2 - s1
        img=np.expand_dims(img, axis=0)
        p= np.squeeze (model.predict(img))           
        index=np.argmax(p)         
        prob=p[index]
        index_list.append(index)
        prob_list.append(prob)
    if image_count==1:
        class_name= class_df['class'].iloc[index_list[0]]
        probability= prob_list[0]
        imgpath=path_list[0]        
        img=plt.imread(imgpath)/255
        plt.title(class_name, color='blue', fontsize=16)
        plt.imshow(img)
        return class_name, probability
    most=0
    for i in range (len(index_list)-1):
        key= index_list[i]
        keycount=0
        for j in range (i+1, len(index_list)):
            nkey= index_list[j]            
            if nkey == key:
                keycount +=1                
        if keycount> most:
            most=keycount
            isave=i             
    best_index=index_list[isave]    
    psum=0
    bestsum=0
    for i in range (len(index_list)):
        psum += prob_list[i]
        if index_list[i]==best_index:
            bestsum += prob_list[i]  
    imgpath=path_list[isave]
    img=plt.imread(imgpath)/255
    class_name=class_df['class'].iloc[best_index]
    plt.title(class_name, color='blue', fontsize=16)
    plt.imshow(img)
    return class_name, bestsum/image_count

In [ ]:
predict_dir=r'../input/butterfly-images40-species/butterflies_rev2/images to predict'  
name='unknown butterfly'
mloc=r'../input/butterfly-images40-species/butterflies_rev2/InceptionResNetV2-butterflies-98.40.h5'
csvloc=r'../input/butterfly-images40-species/butterflies_rev2/class_dict.csv'
result, probability=classify(predict_dir, csvloc,mloc, name)
print (f' {name} is predicted as being {result} with a probability of {probability * 100:5.2f} %')

In [ ]:
predict_dir=r'../input/butterfly-images40-species/butterflies_rev2/single image to predict'
result, probability=classify(predict_dir, csvloc,mloc, name)
print (f' {name} is predicted as being {result} with a probability of {probability * 100:5.2f} %')